# BentoML Example: SKLearn to ONNX model


BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This example notebooks demonstrates how to build SK model and packed as ONNX model for BentoML

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=onnx&ea=onnx-sk-iris-classifier&dt=onnx-sk-iris-classifier)

In [1]:
!pip install -q bentoml "skl2onnx==1.7.0" "onnx==1.7.0" "onnxmltools==1.7.0"

  Using cached skl2onnx-1.7.0-py2.py3-none-any.whl (191 kB)
  Using cached onnxmltools-1.7.0-py2.py3-none-any.whl (252 kB)
  Using cached onnxconverter_common-1.7.0-py2.py3-none-any.whl (64 kB)
  Using cached keras2onnx-1.7.0-py3-none-any.whl (96 kB)
  Using cached fire-0.3.1.tar.gz (81 kB)
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=771d43d8451dfcbda6c1066b6e619b3f1ddec020d8ecc3fb0d8c0e70bbf8e526
  Stored in directory: /home/bentoml/.cache/pip/wheels/95/38/e1/8b62337a8ecf5728bdc1017e828f253f7a9cf25db999861bec
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=4551679660493ab6f79c2ab0add35ed6f744f524e43039ab340d2abc7d796707
  Stored in directory: /home/bentoml/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built fire termcolor


In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [2]:
clr.predict([[5.1, 3.5, 1.4, 0.2]])

array([0])

In [3]:
%%writefile iris_classifier_onnx.py

import numpy
import bentoml
from bentoml.adapters import DataframeInput
from bentoml.frameworks.onnx import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class IrisClassifierOnnx(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        input_data = df.to_numpy().astype(numpy.float32)
        input_name = self.artifacts.model.get_inputs()[0].name
        output_name = self.artifacts.model.get_outputs()[0].name
        outputs = numpy.zeros(input_data.shape[0])
        for i in range(input_data.shape[0]):
            outputs[i] = self.artifacts.model.run([output_name], {input_name: input_data[i: i + 1]})[0]
        return outputs

Writing iris_classifier_onnx.py


In [4]:
from iris_classifier_onnx import IrisClassifierOnnx
from skl2onnx.common.data_types import FloatTensorType
import onnxmltools



initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = onnxmltools.convert_sklearn(clr, initial_types=initial_type)

svc = IrisClassifierOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2020-09-22 14:32:58,569] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 14:32:58,902] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 14:33:01,032] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/setuptools/dist.py:476: UserWarning: Normalizing '0.9.0.pre+3.gcebf2015' to '0.9.0rc0+3.gcebf2015'
  normalized_version,
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 14:33:06,941] INFO - BentoService bundle 'IrisClassifierOnnx:20200922143300_5EB8CF' saved to: /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200922143300_5EB8CF


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [2]:
!bentoml serve IrisClassifierOnnx:latest --enable-microbatch

[2020-08-04 09:58:22,230] INFO - Getting latest version IrisClassifierOnnx:20200804094903_8746D7
[2020-08-04 09:58:22,230] INFO - Starting BentoML API server in development mode..
[2020-08-04 09:58:23,377] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 09:58:23,405] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 09:58:23,406] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.4, but current environment version is 3.6.10.
[2020-08-04 09:58:24,464] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve IrisClassifierOnnx:latest --run-with-ngrok

#### Send prediction requeset to the REST API server

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'
```



## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [6]:
!bentoml containerize IrisClassifierOnnx:latest

[2020-09-22 14:38:45,682] INFO - Getting latest version IrisClassifierOnnx:20200922143300_5EB8CF
Found Bento: /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200922143300_5EB8CF
[2020-09-22 14:38:45,725] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 14:38:45,742] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'irisclassifieronnx:20200922143300_5EB8CF'
Building Docker image irisclassifieronnx:20200922143300_5EB8CF from IrisClassifierOnnx:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200922143300_5EB8CF ['Dockerfile'

/Collecting pytz>=2011k
\Collecting onnx>=1.2.3
-Requirement already satisfied: six in /opt/conda/lib/python3.7/site-packages (from sqlalchemy-utils<0.36.8->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.15.0)
/Requirement already satisfied: async-timeout<4.0,>=3.0 in /opt/conda/lib/python3.7/site-packages (from aiohttp->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (3.0.1)
\Installing collected packages: numpy, pytz, pandas, onnx, onnxruntime
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
/    Uninstalling numpy-1.19.2:
\      Successfully uninstalled numpy-1.19.2
-Successfully installed numpy-1.18.4 onnx-1.7.0 onnxruntime-1.3.0 pandas-0.24.2 pytz-2020.1
\ ---> 866c51f54760
Step 8/15 : COPY . /bento
/ ---> 6e11b086aa71
Step 9/15 : RUN if [ -d /bento/bundled_pip_dependencies ]; then pip install -U bundled_pip_dependencies/* ;fi
 ---> Running in 675f3fde37a6
-Processing ./bundled_pip_dependencies/BentoML-0.9.0rc0+3.gcebf2015.tar.gz


\Requirement already satisfied, skipping upgrade: cerberus in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+3.gcebf2015) (1.3.2)
-Requirement already satisfied, skipping upgrade: six>=1.9 in /opt/conda/lib/python3.7/site-packages (from protobuf>=3.6.0->BentoML==0.9.0rc0+3.gcebf2015) (1.15.0)
/Requirement already satisfied, skipping upgrade: botocore<1.19.0,>=1.18.2 in /opt/conda/lib/python3.7/site-packages (from boto3->BentoML==0.9.0rc0+3.gcebf2015) (1.18.2)
\  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+3.gcebf2015-py3-none-any.whl size=3064091 sha256=b065620707c341cb307a75ee66e4c24eafeb59170d1bc3c344f040dcad703f83
  Stored in directory: /root/.cache/pip/wheels/a0/45/41/62152db705af4ff47e7a3d6abf6247986eef4aa1b94a58d3b9
Successfully built BentoML
\Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.9.0rc0
-    Uninstalling Be

In [7]:
!docker run --rm -p 5000:5000 irisclassifieronnx:20200922143300_5EB8CF

[2020-09-22 21:40:47,259] INFO - Starting BentoML API server in production mode..
[2020-09-22 21:40:47,615] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-22 21:40:47 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-22 21:40:47 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-22 21:40:47 +0000] [1] [INFO] Using worker: sync
[2020-09-22 21:40:47 +0000] [11] [INFO] Booting worker with pid: 11
[2020-09-22 21:40:47 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-22 21:40:47 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-22 21:40:47,877] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-22 21:40:47,888] WARNING - Using BentoML not from official PyPI relea

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [ ]:
from bentoml import load


svc = load(saved_path)

print(svc.predict([[5.1, 3.5, 1.4, 0.2]]))

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [9]:
!bentoml run IrisClassifierOnnx:latest predict --input '[[5.1, 3.5, 1.4, 0.2], [5.1, 3.5, 1.4, 0.2]]'

[2020-09-22 14:41:28,131] INFO - Getting latest version IrisClassifierOnnx:20200922143300_5EB8CF
[2020-09-22 14:41:28,177] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 14:41:28,192] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 14:41:28,584] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 14:41:29,074] INFO - Initializing onnxruntime InferenceSession from onnx file:'/Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200922143300_5EB8CF/IrisClassifierOnnx/artifacts/model.onnx'
[2020-09-22 14:41:33,065] I

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)